<a href="https://colab.research.google.com/github/lucianomartinsjr/agricultura4.0/blob/main/Agricultura4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalações

In [ ]:
!pip install -q -U crewai
!pip install -q -U langchain_google_genai
!pip install -q -U duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 325.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.4/463.4 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

Importando Crew AI

In [ ]:
import os

from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
from google.colab import userdata
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

my_llm = LLM(
              model='gemini/gemini-1.5-flash',
              api_key=os.environ["GEMINI_API_KEY"]
            )

**Ferramentas**

In [ ]:
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

**Agentes**

In [ ]:
buscador = Agent(
    role = 'Agente de Busca e Consulta',
    goal = 'Buscar conteúdo on line sobre informações sobre o clima e condições específicas para uma cultura',
    backstory = '''
          Você é um especialista em agricultura e meteorologia.
          Quero que forneça informações sobre as condições climáticas atuais e previsões para a cultura de {tipo de cultura} na região de {local}.
          Retorne detalhes sobre temperatura, umidade, e outras condições que possam impactar o desenvolvimento dessa cultura.”
                ''',
    llm = my_llm,
    verbose=True,
    allow_delegate=False,
    tools = [search_tool]
)

In [ ]:
fitopatologista = Agent(
    role = 'Especialista para recomendação para Pragas e Doenças',
    goal = 'Fornecer recomendações para o tratamento de pragas e doenças comuns em uma cultura, considerando o clima e as condições locais.',
    backstory = ''''
        Você é um consultor agrícola especializado em fitopatologia.
        Baseado na cultura de {tipo de cultura} e nas condições de umidade e temperatura informadas ([dados de umidade e temperatura]),
        liste as pragas e doenças mais comuns e recomende tratamentos ou práticas preventivas.
        Considere também condições locais como o tipo de solo e histórico de pragas na área.”
                ''',
    tools = [search_tool],
    llm=my_llm,
    verbose=True
)

In [ ]:
editor = Agent(
    role = 'Editor de conteúdo',
    goal = 'Editar um texto de LinkedIn para que o mesmo seja factualmente correto e tenha um tom informal e divertido.',
    backstory = ''''
                Você está trabalhando na edição e correção de um artigo para o LinkedIn.
                Você vai receber um texto do Escritor de Conteúdo e vai revisá-lo e editá-lo para o tom de voz.
                Seu texto para a publicação deve ser em português.
                ''',
    tools = [search_tool],
    llm=my_llm,
    verbose=True
)

**Tarefas**

In [ ]:
buscar = Task(
    description=
        '''
        1. Priorize as últimas tendências, os principais atores e as notícias mais relevantes sobre o {tema}.\n
        2. Identifique o públic-alvo, considerando seus interesses e pontos de dor.\n
        3. Inclua palavras-chave de SEO e o dados de fontes relevantes.
        ''',
    agent=buscador,
    expected_output='Um plano de tendências sobre {tema} com as palavras mais relevantes de SEO e as últimas notícias.'
)

In [ ]:
escrever = Task(
    description='''
        1. Use os dados coletados de conteúdo para criar um post de LinkedIn atraente sobre o {tema}.\n
        2. Incorpore palavras-chave de SEO de forma natural. \n
        3. Certifique-se de que o post esteja estruturado de forma cativante, com uma conclusão que faça o leitor refletir. \n
        ''',
    agent=escritor,
    expected_output='Um texto sobre {tema} para o Linkedin.'
)

In [ ]:
revisar = Task(
    description=("Revisar a postagem de LinkedIn em questão quanto a erros gramaticais"
    ),
    agent=editor,
    expected_output='Um texto de Linkedin pronto para a publicação. O texto está separado em parágrafos e não utiliza bullet point'
)

**Criando Crew (Equipe)**

In [ ]:
crew = Crew(
    agents=[buscador,escritor,editor],
    tasks=[buscar,escrever,revisar],
    verbose=1,
    process=Process.sequential
)

**Executando**

In [ ]:
tema_de_pesquisa = 'Sandro Moreira, organizador do GDG Rio Verde e Professor da UniRV'

entradas = {"tema": tema_de_pesquisa}

resultado = crew.kickoff(inputs=entradas)

# Agent: Buscador de conteúdo
## Task: 
        1. Priorize as últimas tendências, os principais atores e as notícias mais relevantes sobre o Sandro Moreira, organizador do GDG Rio Verde e Professor da UniRV.

        2. Identifique o públic-alvo, considerando seus interesses e pontos de dor.

        3. Inclua palavras-chave de SEO e o dados de fontes relevantes.
        


# Agent: Buscador de conteúdo
## Thought: Thought: I need to find information about Sandro Moreira, his role in GDG Rio Verde, his work as a professor at UniRV, and the latest news and trends related to him.
## Using tool: duckduckgo_search
## Tool Input: 
"{\"query\": \"Sandro Moreira GDG Rio Verde UniRV\"}"
## Tool Output: 
Google Developer Groups GDG Rio Verde presents Build with AI | May 4, 2024. ... UniRV - Universidade de Rio Verde, Rio Verde Campus, Fazenda Fontes do Saber - Campus Administrativo - Bloco IV, Rio Verde, 75901-970 ... Sandro Moreira. Google for Startups. GDG Organizer. See bio. Pollyana Queiro

ERROR:root:LiteLLM call failed: list index out of range


# Agent: Buscador de conteúdo
## Task: 
        1. Priorize as últimas tendências, os principais atores e as notícias mais relevantes sobre o Sandro Moreira, organizador do GDG Rio Verde e Professor da UniRV.

        2. Identifique o públic-alvo, considerando seus interesses e pontos de dor.

        3. Inclua palavras-chave de SEO e o dados de fontes relevantes.
        


# Agent: Buscador de conteúdo
## Thought: Thought: I need to gather information about Sandro Moreira, the GDG Rio Verde, and his role as a professor at UniRV. I'll start by searching for his name on DuckDuckGo to see what comes up.
## Using tool: duckduckgo_search
## Tool Input: 
"{\"query\": \"Sandro Moreira GDG Rio Verde UniRV\"}"
## Tool Output: 
Google Developer Groups GDG Rio Verde presents Build with AI | May 4, 2024. ... UniRV - Universidade de Rio Verde, Rio Verde Campus, Fazenda Fontes do Saber - Campus Administrativo - Bloco IV, Rio Verde, 75901-970 ... Sandro Moreira. Google for Startups. GDG Organizer. 